In [1]:
import pandas as pd
import numpy as np
import sklearn.model_selection
from sklearn.experimental import enable_hist_gradient_boosting
import sklearn.ensemble
import sklearn.metrics
import joblib
from pprint import pprint
np.set_printoptions(precision=2, suppress=True)

In [2]:
class G:
    # global variables
    pass

In [3]:
G.df_train_full = pd.read_csv('X_train', dtype=np.int32)
G.df_test_full = pd.read_csv('X_test', dtype=np.int32)
G.y_train = np.array(open('Y_train').read().strip('\n').split('\n'), dtype=np.int8)

In [4]:
def normalize(df, means, stds):
#     cols = ['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']
    cols = df.columns
    df = df.copy()
    df[cols] = (df[cols] - means[cols]) / stds[cols]
    return df

def extract(df):
    c = len(df.columns)
    n = len(df)
    d = 1 + c #+ 5*5 + 5

    X = np.zeros((n, d), dtype=np.float64)
    X[:, 0] = 1  # bias
    X[:, 1:1+c] = df.values

    # quadratic term (including cross product)
    # ['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']
#     idx = np.array([1, 2, 4, 5, 6])
#     st = 1 + c
#     en = 1 + c + 5*5
#     X[:, st:en] = (X[:, idx[:, None]] * X[:, idx[None, :]]).reshape((n, 5*5))
    
#     # cubic term
#     st = 1 + c + 25
#     en = 1 + c + 25 + 5
#     X[:, st:en] = X[:, idx] ** 3

    return X

def preprocess(df_train, df_test):
    df_all = pd.concat((df_train, df_test))
    means = df_all.mean()
    stds = df_all.std()
    
    df_train = normalize(df_train, means, stds)
    df_test = normalize(df_test, means, stds)
    X_train = extract(df_train)
    X_test = extract(df_test)
    
    print('n,d', X_train.shape)

    return X_train, X_test

G.X_train, G.X_test = preprocess(G.df_train_full, G.df_test_full)
# np.savetxt('a.csv',G.X_train,fmt='%.2f',delimiter=',')

n,d (32561, 107)


In [9]:
clf = sklearn.ensemble.HistGradientBoostingClassifier()
grid = sklearn.model_selection.GridSearchCV(
        clf,
        {
            'learning_rate': np.linspace(1e-3, 1, 20),
            'random_state': [0],
            'l2_regularization': np.linspace(1e-3, 1, 20),
        },
        scoring='accuracy',
        n_jobs=8,
        cv=3,
        return_train_score=True,
        refit=True)
grid.fit(G.X_train, G.y_train)
pprint(grid.cv_results_)

{'mean_fit_time': array([7.09, 7.38, 5.4 , 3.69, 3.25, 3.08, 1.94, 1.71, 1.6 , 1.31, 1.13,
       1.43, 1.41, 1.16, 1.36, 1.03, 1.08, 1.03, 1.11, 1.02, 6.55, 5.64,
       4.55, 3.32, 2.92, 2.18, 2.25, 1.54, 1.29, 1.36, 1.42, 1.04, 1.11,
       1.18, 0.97, 0.84, 1.11, 1.02, 1.  , 0.84, 5.64, 6.41, 5.17, 3.42,
       2.36, 2.44, 1.93, 2.27, 1.89, 1.43, 1.22, 1.67, 1.33, 1.02, 1.05,
       0.99, 1.16, 1.1 , 0.92, 0.88, 5.7 , 5.51, 5.04, 3.05, 3.14, 2.24,
       1.95, 1.69, 1.61, 1.29, 1.45, 1.51, 1.24, 1.16, 1.3 , 1.04, 1.02,
       1.16, 0.98, 1.19, 5.17, 6.21, 5.45, 3.47, 2.74, 2.12, 1.98, 2.02,
       1.31, 1.19, 1.16, 1.14, 1.28, 1.18, 1.4 , 1.25, 1.18, 0.99, 0.91,
       0.87, 5.72, 5.46, 5.31, 3.25, 3.56, 2.1 , 2.55, 1.95, 1.52, 1.43,
       1.49, 1.28, 1.4 , 1.32, 1.18, 1.  , 1.1 , 1.05, 1.22, 1.03, 5.78,
       5.45, 5.13, 3.46, 2.52, 2.67, 1.71, 1.83, 1.65, 1.3 , 1.44, 1.46,
       1.26, 1.27, 1.29, 1.26, 1.04, 1.05, 0.96, 1.02, 5.5 , 5.42, 5.16,
       3.55, 2.95, 2.63, 1.88, 1.

             'learning_rate': 0.6319473684210526,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.6845263157894736,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.7371052631578947,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.7896842105263158,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.8422631578947368,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.8948421052631579,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 0.9474210526315789,
             'random_state': 0},
            {'l2_regularization': 0.3164736842105263,
             'learning_rate': 1.0,
      

             'learning_rate': 0.3164736842105263,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.36905263157894735,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.4216315789473684,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.47421052631578947,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.5267894736842105,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.5793684210526315,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.6319473684210526,
             'random_state': 0},
            {'l2_regularization': 0.8422631578947368,
             'learning_rate': 0.6845263

In [10]:
pprint(grid.best_params_)

{'l2_regularization': 0.36905263157894735,
 'learning_rate': 0.05357894736842105,
 'random_state': 0}


In [12]:
G.y_test = grid.predict(G.X_test)
df_pred = pd.DataFrame({
    'id': np.arange(1, len(G.X_test)+1),
    'label': G.y_test
})
df_pred.to_csv('submission.csv', index=False)
print(df_pred['label'].values[:100])

[0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0]


In [11]:
joblib.dump(grid, 'boost/3.pkl')

['boost/3.pkl']